In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, train_test_split

from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import  Model
from tensorflow.keras.layers import (Conv1D, MaxPooling1D, Dense, Dropout,  BatchNormalization, GlobalAveragePooling1D,   Input, Activation ,SpatialDropout1D, LSTM, Bidirectional)  # ← ADDED LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2
import tensorflow.keras.backend as K
import random

#Load Data
X = np.load(r"preprocessed\ALL_X.npy")
y = np.load(r"preprocessed\ALL_y.npy")

# Convert to Binary Classification 
y_encoded = np.where(y == 'NORMAL', 0, 1)

print("Original classes:", np.unique(y))
print("Binary classes: 0 (NORMAL) vs 1 (INTERICTAL + ICTAL)")
print("Binary labels distribution:", np.unique(y_encoded, return_counts=True))

#  Prepare Data
X = X.reshape((X.shape[0], X.shape[1], 1))
print("Dataset shape:", X.shape)

# Compute Class Weights 
class_weights = compute_class_weight('balanced', classes=np.unique(y_encoded), y=y_encoded)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
print(f"Class weights: {class_weight_dict}")

# -------------------- Prepare Cross Validation --------------------
random_state = np.random.randint(0, 10000)
print(f"🎲 Random state used for this run: {random_state}")

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
fold_indices = [(train_val_idx, test_idx) for train_val_idx, test_idx in kfold.split(X, y_encoded)]

# Save indices for reproducibility
os.makedirs("results", exist_ok=True)
np.save("results/fold_indices.npy", np.array(fold_indices, dtype=object), allow_pickle=True)




def focal_loss(alpha=0.75, gamma=2.0):

    def loss(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        
        # Calculate cross entropy
        cross_entropy = -y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred)
        
        # Calculate focal term
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        focal_term = K.pow(1 - p_t, gamma)
        
        # Calculate alpha weight
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        
        return K.mean(alpha_t * focal_term * cross_entropy)
    
    return loss

# NEW: Hybrid loss for better LSTM training
def hybrid_focal_loss(alpha=0.70, gamma=1.5, focal_weight=0.5):
    """Hybrid: 50% Focal + 50% BCE - Better for LSTM"""
    def loss(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        
        # Binary Cross-Entropy
        bce = -y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred)
        
        # Focal Loss
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        focal_term = K.pow(1 - p_t, gamma)
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        focal = alpha_t * focal_term * bce
        
        # Combine: 50-50
        combined = focal_weight * focal + (1 - focal_weight) * bce
        return K.mean(combined)
    
    return loss


# -------------------- Model Building Function --------------------
def build_improved_model(input_shape):
    inputs = Input(shape=input_shape)
    
    
    # Block 1: Local patterns
    x = Conv1D(48, kernel_size=7, padding='same', kernel_regularizer=l2(0.002))(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = SpatialDropout1D(0.3)(x)
    
    # Block 2: Mid-level features
    x = Conv1D(96, kernel_size=5, padding='same', kernel_regularizer=l2(0.002))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = SpatialDropout1D(0.35)(x)
    
    # Block 3: High-level features
    x = Conv1D(128, kernel_size=3, padding='same', kernel_regularizer=l2(0.002))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = SpatialDropout1D(0.4)(x)
    
    # ========== LSTM Temporal Modeling ========== (SIMPLIFIED - ONE LAYER)
    # Bidirectional LSTM to capture forward and backward temporal patterns
    x = Bidirectional(LSTM(64, return_sequences=False, kernel_regularizer=l2(0.001)))(x)
    x = Dropout(0.4)(x)
    
    # ========== Dense Classification ========== (UNCHANGED)
    x = Dense(64, activation='relu', kernel_regularizer=l2(0.003))(x)
    x = Dropout(0.5)(x)
    
    x = Dense(32, activation='relu', kernel_regularizer=l2(0.003))(x)
    x = Dropout(0.5)(x)
    
    outputs = Dense(1, activation='sigmoid')(x)
    
    return Model(inputs=inputs, outputs=outputs)

# -------------------- Training --------------------
acc_per_fold = []
auc_per_fold = []
conf_matrices = []
class_names = ['NORMAL', 'ALL (INTERICTAL+ICTAL)']

# Track metrics for best model selection
fold_metrics = {
    'fold_no': [],
    'test_acc': [],
    'test_loss': [],
    'test_auc': [],
    'val_acc': [],
    'train_acc': [],
    'f1_score': [],
    'precision': [],
    'recall': []
}
best_model = None
best_fold = None
best_acc = 0

for fold_no, (train_val_idx, test_idx) in enumerate(fold_indices, start=1):
    
    print(f"\n{'='*60}")
    print(f" FOLD {fold_no}")
    print(f"{'='*60}\n")
   

    # Split into train/val/test
    X_train_val, X_test = X[train_val_idx], X[test_idx]
    y_train_val, y_test = y_encoded[train_val_idx], y_encoded[test_idx]

    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=0.15, stratify=y_train_val, random_state=42
    )

    print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")
    print(f"Test set distribution: {np.unique(y_test, return_counts=True)}")

    # Build model
    model = build_improved_model(input_shape=(X.shape[1], 1))
    
    # Compile with LOWER learning rate for LSTM
    model.compile(
        optimizer=Adam(learning_rate=1e-4),  # Changed from 2e-4
        loss=hybrid_focal_loss(alpha=0.70, gamma=1.5, focal_weight=0.5),  # Using hybrid loss
        metrics=['accuracy']
    )
    
    
    
    #  Callbacks 
    early_stop = EarlyStopping(  monitor='val_loss', patience=22, restore_best_weights=True, verbose=1, mode='min')
    
    reduce_lr = ReduceLROnPlateau( monitor='val_loss',factor=0.6,patience=8, min_lr=1e-7,verbose=1,mode='min' )
    
    checkpoint = ModelCheckpoint( f"results/model_fold{fold_no}.weights.h5", monitor='val_accuracy', save_best_only=True, save_weights_only=True,verbose=0,  mode='max')
    
    callbacks = [early_stop, reduce_lr, checkpoint]
    
    # -------------------- Train Model --------------------
    print(f"\n Training Fold {fold_no}...")
    history = model.fit(
        X_train, y_train,
        epochs=150,
        batch_size=32,
        validation_data=(X_val, y_val),
        callbacks=callbacks,
        class_weight=class_weight_dict,
        verbose=1
    )

    # Load best weights
    model.load_weights(f"results/model_fold{fold_no}.weights.h5")

    # -------------------- Evaluate --------------------
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    acc_per_fold.append(test_acc)
    
    # Predictions
    y_pred_prob = model.predict(X_test, verbose=0)
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()
    
    # AUC Score
    test_auc = roc_auc_score(y_test, y_pred_prob)
    auc_per_fold.append(test_auc)
    
    print(f"\n✅ Fold {fold_no} Results:")
    print(f"  Test Accuracy: {test_acc:.4f}")
    print(f"  Test AUC: {test_auc:.4f}")
    print(f"  Test Loss: {test_loss:.4f}")


    #Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    conf_matrices.append(cm)
    
  

    #  Classification Report 
    from sklearn.metrics import precision_recall_fscore_support
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    
    # Store metrics
    fold_metrics['fold_no'].append(fold_no)
    fold_metrics['test_acc'].append(test_acc)
    fold_metrics['test_loss'].append(test_loss)
    fold_metrics['test_auc'].append(test_auc)
    fold_metrics['val_acc'].append(max(history.history['val_accuracy']))
    fold_metrics['train_acc'].append(max(history.history['accuracy']))
    fold_metrics['f1_score'].append(f1)
    fold_metrics['precision'].append(precision)
    fold_metrics['recall'].append(recall)

    
    # Check if this is the best model
    if test_acc > best_acc:
        best_acc = test_acc
        best_fold = fold_no
        best_model = model
        best_cm = cm
        best_y_test = y_test
        best_y_pred = y_pred
        best_y_pred_prob = y_pred_prob
        print(f"🌟 New best model! Fold {fold_no} with accuracy: {test_acc:.4f}")
    
    print(f"\nFold {fold_no} Classification Report:")
    print(classification_report(y_test, y_pred, target_names=class_names, digits=4))


    #  Plot Confusion Matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                annot_kws={'size': 14})
    plt.title(f"Fold {fold_no} Confusion Matrix\nAcc: {test_acc:.4f} | AUC: {test_auc:.4f}", 
              fontsize=14)
    plt.xlabel("Predicted", fontsize=12)
    plt.ylabel("True", fontsize=12)
    plt.tight_layout()
    plt.savefig(f"results/confusion_fold{fold_no}.png", dpi=300)
    plt.close()

 

    #  Plot Training History 
    plt.figure(figsize=(14, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    plt.plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
    plt.axhline(y=test_acc, color='r', linestyle='--', label=f'Test Acc: {test_acc:.4f}')
    plt.title(f'Fold {fold_no} - Model Accuracy', fontsize=13)
    plt.xlabel('Epoch', fontsize=11)
    plt.ylabel('Accuracy', fontsize=11)
    plt.legend(fontsize=10)
    plt.grid(alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss', linewidth=2)
    plt.plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    plt.title(f'Fold {fold_no} - Model Loss', fontsize=13)
    plt.xlabel('Epoch', fontsize=11)
    plt.ylabel('Loss', fontsize=11)
    plt.legend(fontsize=10)
    plt.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f"results/training_history_fold{fold_no}.png", dpi=300)
    plt.close()

# -------------------- Summary --------------------
print("\n" + "="*60)
print(" CROSS-VALIDATION SUMMARY")
print("="*60)
print(f"\n📊 Mean Test Accuracy across folds: {np.mean(acc_per_fold):.4f}")
print(f"📊 Mean Test AUC across folds: {np.mean(auc_per_fold):.4f}")

# Combine confusion matrices
total_cm = np.sum(conf_matrices, axis=0)
plt.figure(figsize=(8, 6))
sns.heatmap(total_cm, annot=True, fmt='d', cmap='Greens', 
            xticklabels=class_names, yticklabels=class_names,
            annot_kws={'size': 14})
plt.title("Overall Confusion Matrix (All Folds)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.savefig("results/confusion_overall.png", dpi=300)
plt.close()

# -------------------- Print Fold Metrics --------------------
print("\n📋 Fold-wise Metrics Summary:")
for i in range(len(fold_metrics['fold_no'])):
    print(f"\n🔸 Fold {fold_metrics['fold_no'][i]} Metrics:")
    print(f"  Train Accuracy : {fold_metrics['train_acc'][i]:.4f}")
    print(f"  Val Accuracy   : {fold_metrics['val_acc'][i]:.4f}")
    print(f"  Test Accuracy  : {fold_metrics['test_acc'][i]:.4f}")
    print(f"  Test Loss      : {fold_metrics['test_loss'][i]:.4f}")
    print(f"  Precision      : {fold_metrics['precision'][i]:.4f}")
    print(f"  Recall         : {fold_metrics['recall'][i]:.4f}")
    print(f"  F1 Score       : {fold_metrics['f1_score'][i]:.4f}")
    print(f"  Test AUC       : {fold_metrics['test_auc'][i]:.4f}")

# Save best model
best_model.save("results/best_model.keras")
print(f"\n💾 Best model (Fold {best_fold}) saved as 'results/best_model.keras'")

print("\n✅ Training completed! All weights, folds, and confusion matrices saved in 'results/' folder.")
print(f"\n🎯 BEST MODEL: Fold {best_fold} with Test Accuracy = {best_acc:.4f}")

Original classes: ['ICTAL' 'INTERICTAL' 'NORMAL']
Binary classes: 0 (NORMAL) vs 1 (INTERICTAL + ICTAL)
Binary labels distribution: (array([0, 1]), array([4400, 6600], dtype=int64))
Dataset shape: (11000, 347, 1)
Class weights: {0: 1.25, 1: 0.8333333333333334}
🎲 Random state used for this run: 9892

 FOLD 1

Train: 7480, Val: 1320, Test: 2200
Test set distribution: (array([0, 1]), array([ 880, 1320], dtype=int64))

 Training Fold 1...
Epoch 1/150
234/234 ━━━━━━━━━━━━━━━━━━━━ 28s 82ms/step - accuracy: 0.5369 - loss: 1.4756 - val_accuracy: 0.6000 - val_loss: 1.3988 - learning_rate: 1.0000e-04
Epoch 2/150
 96/234 ━━━━━━━━━━━━━━━━━━━━ 10s 73ms/step - accuracy: 0.5673 - loss: 1.4112

KeyboardInterrupt: 